In [48]:
import numpy as np
from random import sample

from util import read_horoscopes
from keras.models import Sequential
from keras.layers import LSTM, GRU, TimeDistributed, Activation, Dense, Dropout

In [3]:
HIDDEN_DIM = 100
LAYER_NUM = 3
SEQ_LENGTH = 100
BATCH_SIZE=128
GENERATE_LENGTH=100

START_TOKEN = '<'
END_TOKEN = '>'

In [4]:
data = read_horoscopes(tokenize=True).split('\n')

In [5]:
chars = set(' '.join(data) + START_TOKEN + END_TOKEN)
ix_to_char = {ix: char for ix, char in enumerate(chars)}
char_to_ix = {char: ix for ix, char in enumerate(chars)}

VOCAB_SIZE = len(chars)

In [15]:
X = np.zeros((len(data), SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((len(data), SEQ_LENGTH, VOCAB_SIZE))
for i in range(0, len(data)):
    X_sequence = data[i][:SEQ_LENGTH-2]
    X_sequence = X_sequence + (' ' * ((SEQ_LENGTH-2) - len(X_sequence)))
    X_sequence = START_TOKEN + X_sequence + END_TOKEN
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = data[i][1:SEQ_LENGTH-1]
    y_sequence = y_sequence + (' ' * ((SEQ_LENGTH-2) - len(y_sequence)))
    y_sequence = START_TOKEN + y_sequence + END_TOKEN
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

In [16]:
model = Sequential()
model.add(GRU(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(GRU(HIDDEN_DIM, return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_5 (GRU)                  (None, None, 100)         56100     
_________________________________________________________________
gru_6 (GRU)                  (None, None, 100)         60300     
_________________________________________________________________
gru_7 (GRU)                  (None, None, 100)         60300     
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 86)          8686      
_________________________________________________________________
activation_3 (Activation)    (None, None, 86)          0         
Total params: 185,386
Trainable params: 185,386
Non-trainable params: 0
_________________________________________________________________


In [47]:
np.array(chars)

array({'л', 'с', 'И', 'ъ', '…', '–', 'ф', '4', 'х', 'т', 'Ш', 'Щ', 'е', 'щ', 'П', 'ж', '»', 'э', 'ч', ' ', 'Л', 'р', 'з', 'ю', 'Ю', 'С', ';', 'Ч', 'М', 'я', '‑', '«', 'в', 'У', '5', 'ш', 'й', 'ь', 'у', 'Э', 'н', 'Х', '1', '0', '2', '?', ',', '7', 'п', '6', '>', 'К', ')', 'Д', ':', '(', 'Г', '<', 'и', 'Ж', '9', '.', 'Ц', 'г', 'д', 'Б', 'Т', 'ц', 'б', '8', 'о', 'О', '3', 'к', 'м', '!', 'В', 'ы', 'А', 'а', 'Н', 'Р', 'Я', 'Е', 'З', 'Ф'},
      dtype=object)

In [115]:
def generate_text(model, length, start_char=START_TOKEN):
    ix = [char_to_ix[start_char]]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, VOCAB_SIZE))
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        next_ix = np.random.choice(np.arange(VOCAB_SIZE), p=model.predict(X[:, :i+1, :])[0][-1])
        ix.append(next_ix)
        y_char.append(ix_to_char[next_ix])
    return ''.join(y_char)

In [121]:
# Load pretrained model
model.load_weights('checkpoint_100_epoch_500.hdf5')

In [20]:
# Training
nb_epoch = 0
while True:
    print('\n\n')
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=5)
    nb_epoch += 5
    generate_text(model, GENERATE_LENGTH)
    if nb_epoch % 100 == 0:
        model.save_weights('checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, nb_epoch))




Epoch 1/5
4380/4380 [==============================] - 59s - loss: 3.3594    
Epoch 2/5
4380/4380 [==============================] - 55s - loss: 3.1737    
Epoch 3/5
4380/4380 [==============================] - 56s - loss: 2.8113    
Epoch 4/5
4380/4380 [==============================] - 55s - loss: 2.4878    
Epoch 5/5
4380/4380 [==============================] - 55s - loss: 2.2651    
<<< аряята . Пенедень . Вородений . Вородений , нодения . Вородений , нодения , предений , денодения


Epoch 1/5
4380/4380 [==============================] - 55s - loss: 2.1127    
Epoch 2/5
4380/4380 [==============================] - 55s - loss: 1.9857    
Epoch 3/5
4380/4380 [==============================] - 55s - loss: 1.8766    
Epoch 4/5
4380/4380 [==============================] - 55s - loss: 1.7811    
Epoch 5/5
4380/4380 [==============================] - 55s - loss: 1.7019    
<< аютя . Воскресенье . День проделания слагоприятный день . Ворожения слагоприятный день . Ворожени


Epoch 1/5
4

4380/4380 [==============================] - 55s - loss: 0.7168    
Epoch 5/5
4380/4380 [==============================] - 55s - loss: 0.7132    
<<0 июля . Воскресенье . День благоприятен для коллективных проблем , получения или неожиданных пово


Epoch 1/5
4380/4380 [==============================] - 55s - loss: 0.7102    
Epoch 2/5
4380/4380 [==============================] - 55s - loss: 0.7073    
Epoch 3/5
4380/4380 [==============================] - 57s - loss: 0.7044    
Epoch 4/5
4380/4380 [==============================] - 55s - loss: 0.7016    
Epoch 5/5
4380/4380 [==============================] - 55s - loss: 0.6985    
<< мая . Пятница . День отмечен советуют выходности , не следует вам не следует проявить свои против


Epoch 1/5
4380/4380 [==============================] - 55s - loss: 0.6962    
Epoch 2/5
4380/4380 [==============================] - 55s - loss: 0.6935    
Epoch 3/5
4380/4380 [==============================] - 55s - loss: 0.6902    
Epoch 4/5
4380/4380 [===

4380/4380 [==============================] - 55s - loss: 0.5591    
Epoch 2/5
4380/4380 [==============================] - 55s - loss: 0.5586    
Epoch 3/5
4380/4380 [==============================] - 55s - loss: 0.5570    
Epoch 4/5
4380/4380 [==============================] - 55s - loss: 0.5557    
Epoch 5/5
4380/4380 [==============================] - 55s - loss: 0.5570    
<<0 августа . Воскресенье . День благоприятен для общения , новых знакомств , обмана , ложных обещан


Epoch 1/5
4380/4380 [==============================] - 55s - loss: 0.5524    
Epoch 2/5
4380/4380 [==============================] - 55s - loss: 0.5538    
Epoch 3/5
4380/4380 [==============================] - 55s - loss: 0.5537    
Epoch 4/5
4380/4380 [==============================] - 56s - loss: 0.5497    
Epoch 5/5
4380/4380 [==============================] - 56s - loss: 0.5483    
<<0 августа . Воскресенье . День благоприятен для покупок , проявлять силы . Вероятно , сегодня возм


Epoch 1/5
4380/4380 [===

4380/4380 [==============================] - 55s - loss: 0.4959    
<<0 ноября . Четверг . День отмечен обещает быть любые представления о полног . Возможно , что в это


Epoch 1/5
4380/4380 [==============================] - 54s - loss: 0.4946    
Epoch 2/5
4380/4380 [==============================] - 55s - loss: 0.4952    
Epoch 3/5
4380/4380 [==============================] - 55s - loss: 0.4926    
Epoch 4/5
4380/4380 [==============================] - 55s - loss: 0.4940    
Epoch 5/5
4380/4380 [==============================] - 55s - loss: 0.4930    
<<0 ноября . Четверг . Сегодня возможны встречи со стороны постороннее влияние , личное обаяние и со


Epoch 1/5
4380/4380 [==============================] - 54s - loss: 0.4923    
Epoch 2/5
4380/4380 [==============================] - 55s - loss: 0.4916    
Epoch 3/5
4380/4380 [==============================] - 55s - loss: 0.4933    
Epoch 4/5
4380/4380 [==============================] - 55s - loss: 0.4898    
Epoch 5/5
4380/4380 [===

4380/4380 [==============================] - 57s - loss: 0.4617    
Epoch 3/5
4380/4380 [==============================] - 57s - loss: 0.4599    
Epoch 4/5
4380/4380 [==============================] - 57s - loss: 0.4604    
Epoch 5/5
4380/4380 [==============================] - 57s - loss: 0.4602    
<<0 ноября . Четверг . День символизирует разочарования , получение новых серьез до силы , сегодня в


Epoch 1/5
4380/4380 [==============================] - 57s - loss: 0.4613    
Epoch 2/5
4380/4380 [==============================] - 57s - loss: 0.4590    
Epoch 3/5
4380/4380 [==============================] - 57s - loss: 0.4614    
Epoch 4/5
4380/4380 [==============================] - 58s - loss: 0.4594    
Epoch 5/5
4380/4380 [==============================] - 58s - loss: 0.4602    
<<0 ноября . Четверг . День благоприятный для получения сокровенных знаний , покупок , преобразовани


Epoch 1/5
4380/4380 [==============================] - 57s - loss: 0.4580    
Epoch 2/5
4380/4380 [===

<<0 ноября . Четверг . Сегодня вам представится возможность прикум и соблазны , не назначать силы со


Epoch 1/5
4380/4380 [==============================] - 55s - loss: 0.4373    
Epoch 2/5
4380/4380 [==============================] - 55s - loss: 0.4376    
Epoch 3/5
4380/4380 [==============================] - 55s - loss: 0.4361    
Epoch 4/5
4380/4380 [==============================] - 55s - loss: 0.4368    
Epoch 5/5
4380/4380 [==============================] - 55s - loss: 0.4385    
<<0 ноября . Четверг . День благоприятный для получения сокровенных знаний , постижения истин , путе


Epoch 1/5
4380/4380 [==============================] - 55s - loss: 0.4360    
Epoch 2/5
4380/4380 [==============================] - 55s - loss: 0.4356    
Epoch 3/5
4380/4380 [==============================] - 54s - loss: 0.4353    
Epoch 4/5
4380/4380 [==============================] - 55s - loss: 0.4352    
Epoch 5/5
4380/4380 [==============================] - 55s - loss: 0.4364    
<<0 ноября . Ч

4380/4380 [==============================] - 55s - loss: 0.4189    
Epoch 4/5
4380/4380 [==============================] - 55s - loss: 0.4202    
Epoch 5/5
4380/4380 [==============================] - 54s - loss: 0.4195    
<<0 ноября . Воскресенье . День символизирует серьезных предприятное или представления о ситуация . 


Epoch 1/5
4380/4380 [==============================] - 55s - loss: 0.4196    
Epoch 2/5
4380/4380 [==============================] - 55s - loss: 0.4172    
Epoch 3/5
4380/4380 [==============================] - 55s - loss: 0.4194    
Epoch 4/5
4380/4380 [==============================] - 55s - loss: 0.4196    
Epoch 5/5
4380/4380 [==============================] - 55s - loss: 0.4188    
<<0 ноября . Четверг . День неблагоприятный для общения с сотрудниками или начальством , равовести п


Epoch 1/5
4380/4380 [==============================] - 55s - loss: 0.4191    
Epoch 2/5
4380/4380 [==============================] - 55s - loss: 0.4185    
Epoch 3/5
4380/4380 [===

KeyboardInterrupt: 

In [128]:
generate_text(model, 200);

<<0 ноября . Воскресенье . Тяжелый и опасный день . Вероятны проблемы с обманами , заключите все своестансе будет положиться отнишения поездки и оприметре и неожиданных перемен , профессий , особенно 

Источник: https://chunml.github.io/ChunML.github.io/project/Creating-Text-Generator-Using-Recurrent-Neural-Network/